Every git commit needs user and email, so configure your git with -

```
git config --global user.name "abc"
git config --global user.email "xyz@blah.com"
```

To check your settings:
`git config --list`

---

### IAM: User, Group, Role and Policy

Main actors in IAM are users, groups, roles and policies. And what you need to understand about AWS and never forget is that

> Everything in AWS is an API

And to execute any API or any of its methods, first we have to authenticate and then authorize that particular user/group/role.

Ex: An operator wants to put an object to a S3 bucket. This process happens through a set of API calls within AWS. Basically we call the S3 API and a method of it to put the object into the particular bucket (say method `put_object_in_s3`). For that we may want to provide the name of the bucket, the object, and most importantly we need to provide set of credentials (username with password or secret key or etc) in order to tell the AWS API Engine who this user/group/role is.

The first thing API Engine does is, look at those credentials sent with the API. Then it validate those (whether they are correct, active) credentials indicating that this request is coming from a actual valid user, group or role. Then what the API Engine does is (as it now knows who sent this API request) it takes the policy documents associated with the particular operator (user or role) and evaluate them as a single view. That is we check whether the action called in the API is authorized for that operator.

__IAM user__ - In the context of IAM, an user is a “permanent” named operator (human or machine). What’s important to note is that it’s credentials (credentials maybe username password or access key or a secret key) are permanent and stays with that named user. So by that AWS knows that what are the authentication methods (username password authentication method or secret key method or etc) for this user (as its permanent and stays with the user).

__IAM group__ - As in the above image, a group is a collection of users. And note that a user can be in many groups as well.

__IAM roles__ - Roles are not Permissions !!!. A role is also an authentication method just as IAM users and groups. As an user, a role is also a operator (could be a human, could be a machine). Difference is that credentials with roles are temporary.

__Policy Documents__ - As stated earlier, roles are not Permissions. Permissions in AWS are completely handled by objects called `Policy Documents`. Policy Documents are JSON documents. Policy Documents can directly be attached to Users, Groups or Roles. When a policy document gets attached to any of above operator, then only they get permissions do stuff. A policy document lists things like: Specific API or wildcard group of APIs that gets whitelisted against which resources, and Conditions for those API executions (like allow only if this user, group or role in the home network or allow from any location, allow only at certain times of day and etc)

> Last but not least, Authentication in AWS is done via (IAM users, groups and roles) whereas Authorization is done by Policies.

---

### Difference between Region, Availability Zone (AZ) and Edge Location

- A region is a __physical location in the world__ consisting to 2 or more AZs
- An AZ is one or more discrete __data centers__, each with redundant power, networking and connectivity, housed in separate facilities
- Edge Locations are end points for AWS which are used for __caching content__. Typically, this consists of CloudFront, Amazon's Content Delivery Network (CDN)

---

### Key Concepts:

A __Virtual Private Cloud (VPC)__ is a virtual network dedicated to a single AWS account. It is logically isolated from other virtual networks in the AWS cloud, providing compute resources with security and robust networking functionality.

  #### #of Edge Locations > # of Availability Zones > # of Regions

Some of the important services: Storage, Compute, Database, Network and Security, Identity and Roles

> Whenever we are doing anything with IAM the region is set to Global.

---

### IAM

- It is universal
- ROOT account has GOD mode. Make MFA (Multi Factor Authentication) for root user.

- **USER**
    - Two types of access:
        1. Programmatic access (access key ID and secret access key)
        2. Console access (password)
    - At the beginning, user gets no permission/authorization. We need to give permission via policy document. For example, we can give password change related policy (`IAMUserChangePassword`) to the user so that it can change its password.
    - We can add the user to group(s). Group can have different permissions/policies assigned and those policies are automatically inherited to the user under that group.
    - Set __password policy__ for the user in account settings.

### BILLING ALARM (CLOUD WATCH)

We can use SNS (SIMPLE NOTIFICATION SERVICE) under Cloud Watch to automatically send us the notification if the bill exceeds the threshold set by us.

### S3 (Simple Storage Service)

- It is **object-based** i.e allows you to upload files
- Files can be from 0 to 5 TB
- There is unlimited storage
- Files are stored in **buckets** (folders)
- S3 is a universal namespace (names must be unique globally). The region changes to `Global` when on S3 similar to IAM
- http 200 code if the upload was successful

**s3 objects** consists of following -
* key (name of the object)
* value (this is simply the data and is made up of bytes)
* version id (important for versioning)
* metadata (info about the data)
* subresources
    * access control lists (access/permissions (e.g lock an object) on the bucket level as well as individual object level)
    * torrent
    
**s3 data consistency model**
* Read after write consistency for PUTS of new objects (as soon as you create, you'll be able to read that object)
* Eventual consistency for overwrite PUTS and DELETES (might take some time to reflect the change)

**s3 storage classes or access tier**
1. s3 standard (4 9s availability i.e 99.99% availability; 11 9s durability)
2. s3 IA (infrequency accessed) e.g if we access something at the end of every month
    - Lower fee but retrieval charge
3. s3 One Zone IA (similar to deprecated RRS i.e reduced redundancy service) 
    - We do not care about losing the data if something happens as only 1 AZ
4. s3 Intelligent Tiering
    - Uses machine learning to analyse data usage and automatically moves data to most cost effective access tier or storage class to reduce cost
5. s3 glacier
    - for data archival
    - we can configure retrieval time (between minutes and hours)
6. s3 glacier deep archive
    - lowest cost storage class (which can go upto 12 hours for data retrieval)
    
> Note: Read s3 FAQs as s3 is very important for exam

#### S3 storage expense wise

s3 standard > s3 IA > s3 IT > s3 one zone IA > s3 glacier > s3 glacier deep archive

### S3 basics

#### Access control
To setup **access control** to S3 we can use -
1. Bucket Policy - works on bucket level
2. Access Control List (ACL) - works on individual object level

We can configure s3 bucket to log access requests. This log can be sent to another bucket and even another bucket in different AWS account.

#### Encryption
- `Encryption in transit` (HTTPS encryption) is achieved by
    - SSL/TLS
    
- `Encryption at rest (Server Side)` is achieved by
    - S3 Managed Keys (__SSE-S3__: Server Side Encryption): here Amazon manages the key
    - AWS Key Management Service (__SSE-KMS__): here we and Amazon together manage the keys
    - Server Side Encryption with Customer Provided Keys (__SSE-C__): here we give Amazon our own keys
    
- `Encryption at rest (Client Side)`
    - where we encrypt the object at our end to put it in S3
    
#### Versioning S3
- Great backup tool
- Once enabled on bucket, cannot be disabled but can only be suspended.
- For deletion of bucket, we can enable MFA (multi factor authentication)
- Integrates with Lifecycle rules

SCENARIOS:

> If we upload the same file again with some changes, the file gets overwritten and the permission (if you have made the previous uploaded file public) is private (default behaviour). So, we need to make it public again. The first version will still be public and can be seen under "Version Show" button. And when we delete the object, we can still see the different versions and the latest one would have "delete marker" on it. If we delete the "delete marker" object from version table, then it gets removed from the stack and the second last gets to top and becomes the latest revision.

> On overriding the same file, size of it increases due to version control. So keep in mind if you are updating huge file then the size will increase exponentially. In that case, you might want to look into Lifecyle rules.

#### Lifecycle Rule

We can create lifecycle rule configuration that is applicable for whole bucket or for specific tags corresponding to objects within the bucket.

There are two types of actions we can take on bucket/objects:

1. Automatic **transition** to tiered storage (For example, you might choose to transition objects to the S3 Standard-IA storage class 30 days after you created them, or archive objects to the S3 Glacier storage class one year after creating them).
2. **Expire** your objects (Define when objects expire. Amazon S3 deletes expired objects on your behalf).

> Both the actions gets triggered (once enabled) after "N" number of days of the object creation.

We use lifecycle rule when - 

1. If you upload periodic logs to a bucket, your application might need them for a week or a month. After that, you might want to delete them.

2. Some documents are frequently accessed for a limited period of time. After that, they are infrequently accessed. At some point, you might not need real-time access to them, but your organization or regulations might require you to archive them for a specific period. After that, you can delete them.

3. You might upload some types of data to Amazon S3 primarily for archival purposes. For example, you might archive digital media, financial and healthcare records, raw genomics sequence data, long-term database backups, and data that must be retained for regulatory compliance.

#### Object LOCK (Locking objects for future edits/delete for regulations/ critical data etc.)

- We can enable `Object Lock` only when **VERSIONING** in enabled for a bucket.
- We use Object Lock to enable WORM mode i.e Write Once Read Many. This is done to avoid people from overwriting or deleting version of an object. So you put one version and don't want it to be changed or deleted for some time (retention period or indefinitely with legal hold)
- There's lot of combinations that can be put on Object Lock and to read about those check https://docs.aws.amazon.com/AmazonS3/latest/dev/object-lock-overview.html which has a nice documentation.
- Similary, we have Glacier Vault Lock Policy for WORM mode and once locked, the policy cannot be changed.

#### S3 PERFORMANCE

1. **PREFIX**: Bare in mind while designing your architecture/bucket that more prefix you have the better performance we achieve.
   Example of prefix: mybucketname/folder1/subfolder1/abc.jpg > **/folder1/subfolder1**
2. **SSE-KMS**: S3 performance apart from prefix depends on **SSE-KMS**. So if our bucket/object uses that encryption then it will use KMS quota which depending on region can be 5500, 10000 or 30000 requests per second. This is because for example when we upload a file, we will call `GenerateDataKey` in the KMS API and similary while downloading the `Decrypt` key.
3. **MULTIPART UPLOADS**: Recommended for files > 100MB and necessary for files > 5GB
4. **BYTE RANGE FETCHES**: Parallelize byte range downloads

> See Exam Tips of Chapter 18 for # of request per prefix and for SSE-KMS

#### S3 SELECT and GLACIER SELECT

- to retrieve only subset of data. For example if we have a zip file containing CSV files and we want to get only one file then instead of **fetching** the entire zip, **decompressing** it and then reading the actual file, we can use S3 select to query just that file. It will save money for data transfer as well as increase speed.

#### SHARE BUCKET

There are 3 different ways to share S3 buckets -

1. Using Bucket Policy and IAM (bucket level): Programmatic access only
2. Using Bucket ACLs and IAM (object level): Programmatic access only
3. Cross account IAM Roles: Console and Programmatic access

#### CROSS REGION REPLICATION (CRR) OF S3 BUCKET (put in one bucket; automatically in another as well)

* Not that important (understand just the high level)
* Versioning must be enabled on the source bucket for CRR
* Files already existing in the source bucket are automatically replicated
* All subsequent updates will be automatically updated
* Delete markers and delete individual versions are not replicated
* Changing access of objects from private to public in source bucket doesn't impact the object in destination bucket.

#### TRANSFER ACCELERATION (to transfer files faster)

Amazon S3 Transfer Acceleration enables fast, easy, and secure transfers of files over long distances between your client and an S3 bucket. Transfer Acceleration takes advantage of Amazon CloudFront’s globally distributed edge locations. As the data arrives at an edge location, data is routed to Amazon S3 over an optimized network path.

> When using Transfer Acceleration, additional data transfer charges may apply.

Why you might want it?

* You have customers that upload to a centralized bucket from all over the world.
* You transfer gigabytes to terabytes of data on a regular basis across continents.

#### DATA SYNC (like between on-premise server and AWS)

* Used to move large amounts of data from on-premise data center to AWS and vice versa
* Need datasync agent on the source to transfer data
* Used with **NFS** and **SMB** compatible file system
* Replication can be done hourly, daily or weekly
* Can be used to replicate EFS to EFS as well

#### CLOUD FRONT (for delivering content)

* CloudFront is basically a CONTENT DELIVERY NETWORK (CDN)
* CDN is a system of distributed servers that delivers content to a user based on location etc.
* KEY TERMINOLOGIES:
    - ORIGIN: This is the origin of all files that CDN will distribute. This can be either S3 bucket, EC2 instance, Elastic Load Balancer, or Route53
    - DISTRIBUTION: It's the name (domain name) given to CDN which is a collection of edge locations
    - Web Distributino: Typically used for websites
    - RTMP (Real Time Messaging Protocol): For Media Streaming

> Edge Locations are not just READ only. We can write them to (i.e put objects on them). This we saw in Transfer Acceleration.

> Objects are cached for __TTL__ (Time To Live) value.

> You can clear cached object but that will be charged. Example, if we have uploaded something and the users are still getting the old video rather than the new one, then you can go in and probably clear those cached data.

Doc: https://docs.aws.amazon.com/AmazonCloudFront/latest/DeveloperGuide/Introduction.html

#### CLOUD FRONT SIGNED URLs and COOKIES (e.g for premium users)

* Use signed URLs/Cookies when you want to secure content so that only people who authorize can use it.
* Signed URL is for single file. **1 file = 1 URL**
* Signed Cookie is for multiple files. **1 cookie = multiple files**
* If your origin is EC2, use CloudFront. If your origin is S3 and you have single file for user when you can use **S3 signed URL**

#### SNOWBALL (Migration of huge data)
* Import to S3, export to S3. Your requirement for snowball can depend on the size of file as well as available internet connection. For example if you have 2TB data with 44Mbps connection then you migth use it.

#### Storage Gateway (move backups to the cloud)

Scroll to the bottom in the below URL for better info.

Doc: https://aws.amazon.com/storagegateway/?nc=sn&loc=0&whats-new-cards.sort-by=item.additionalFields.postDateTime&whats-new-cards.sort-order=desc

#### ATHENA vs MACIE

ATHENA:
- It is an interactive query service i.e allows you to query on data stored in S3
- It is serverless
- Commonly used to **analyse log data** stored in S3

MACIE:
- It is a security service that uses AI to analyze data stored in S3 and helps **identify PII** (Personal Identifiable Information)
- Can also be used to analyse CloudTrail logs data for suspicious activity
- Includes dashboard, report and alerting

> _TIP: READ S3 FAQs in aws as S3 is really really an important topic for Associate Exams_

#### QUIZ ON S3

1. *Power user access allows access to all AWS services except management of users and groups within IAM*

2. You are a solutions architect working for a large engineering company that are moving from a legacy infrastructure to AWS. You have configured the company's first AWS account and you have set up IAM. Your company is based in Andorra, but there will be a small subsidiary operating out of South Korea, so that office will need its own AWS environment. Which of the following statements is true?
    - Correct Answer: You will need to configure **Users and Policy Documents only once**, as they are applied globally. (Here remember that IAM is GLOBAL)

3. You have created a new AWS account for your company, and you have also configured multi-factor authentication on the root account. You are about to create your new users. What strategy should you consider in order to ensure that there is good security on this account.
    - Correct Answer: Enact a **strong password policy**. user passwords must be changed every 45  days, with each password containing a combination of uppercase, numbers, special characters.
    
4. You have been asked to advise on a scaling concern. The client has an elegant solution that works well. As the information base grows they use CloudFormation to spin up another stack made up of an S3 bucket and supporting compute instances. The trigger for creating a new stack is when the PUT rate approaches 100 PUTs per second. The problem is that as the business grows that number of buckets is growing into the hundreds and will soon be in the thousands. You have been asked what can be done to reduce the number of buckets without changing the basic architecture.
    - Correct Answer: Change the trigger level to around 3500 PUTS as S3 can now accommodate much higher PUT and GET levels.
    - Explanation: Until 2018 there was a hard limit on S3 puts of 100 PUTs per second. To achieve this care needed to be taken with the structure of the name Key to ensure parallel processing. As of July 2018 the limit was raised to 3500 and the need for the Key design was basically eliminated. Disk IOPS is not the issue with the problem. The account limit is not the issue with the problem.
    
5. You run a meme creation website where users can create memes and then download them for use on their own sites. The original images are stored in S3 and each meme's metadata in DynamoDB. You need to decide upon a low-cost storage option for the memes, themselves. If a meme object is unavailable or lost, a Lambda function will automatically recreate it using the original file from S3 and the metadata from DynamoDB. Which storage solution should you use to store the non-critical, easily reproducible memes in the most cost-effective way?
    - Correct Answer: S3-1Z-IA
    - Explanation: S3 – OneZone-IA is the recommended storage for when you want cheaper storage for infrequently accessed objects. It has the same durability but less availability. There can be cost implications if you use it frequently or use it for short lived storage. Glacier is cheaper, but has a long retrieval time. RRS has effectively been deprecated. It still exists but is not a service that AWS want to sell anymore.
    
6. What is the availability of S3 – OneZone-IA?
    - Correct Answer: 99.50%
    - Explanation: OneZone-IA is only stored in one Zone. While it has the same Durability, it may be less Available than normal S3 or S3-IA
    
7. One of your users is trying to upload a 7.5GB file to S3. However, they keep getting the following error message: "Your proposed upload exceeds the maximum allowed object size.". What solution to this problem does AWS recommend?
    - Correct Answer: Design your application to use Multi-part upload API for all objects.
    - Explanation: multipart recommended for more than 100 mb and necessary for 5GB or more. Also, 5 TB max upload size for a file is to be remembered.
    
8. AWS S3 has four different URLs styles that it can be used to access content in S3.  The Virtual Hosted Style URL, the Path-Style Access URL, the Static web site URL, and the Legacy Global Endpoint URL.  Which of these represents a correct formatting of the  Virtual Hosted Style URL  style
    - Correct Answer: https://bucket-name.s3.Region.amazonaws.com/abc.png
    - Explanation:
        Virtual Hosted: https://bucket-name.s3.Region.amazonaws.com/key_name
        
        Path Style: https://s3.Region.amazonaws.com/bucket-name/key_name
        
        Virtual style puts your bucket name 1st, s3 2nd, and the region 3rd.
        
        Path style puts s3 1st and your bucket as a sub domain.
        
        Legacy Global endpoint has no region.
        
        S3 static hosting can be your own domain or your bucket name 1st, s3-website 2nd, followed by the region.
        
        AWS are in the process of phasing out Path style, and support for Legacy Global Endpoint format is limited and discouraged. However it is still useful to be able to recognize them should they show up in logs. https://docs.aws.amazon.com/AmazonS3/latest/dev/VirtualHosting.html
        
9. How many S3 buckets can I have per account by default?
    - Correct Answer: 100
    
10. What is the availability of objects stored in S3?
    - Correct Answer: 99.99%

11. What is AWS Storage Gateway?
    - Correct Answer: It is a physical or virtual appliance that can be used to cache S3 locally at customer's site.
    - Explanation: At its heart it is a way of using AWS S3 managed storage to supplement on-premise storage. It can also be used within a VPC in a similar way.



## QUESTIONS/DOUBTS
